In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/Shareddrives/Dataset_ShapeNetCore/data/
%cd /content/drive/Shareddrives/Deep Elastics/Shen Projects/Github/DeepSDF

/content/drive/Shareddrives/Deep Elastics/Shen Projects/Github/DeepSDF


In [ ]:
!pip3 install mesh-to-sdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import os
import json
import deep_sdf
import logging
from mesh_to_sdf import mesh_to_sdf, sample_sdf_near_surface
import trimesh
import pyrender


/home/shen/miniconda3/envs/pytorch3d/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [3]:
split_filename = 'examples/splits/sv2_sofas_train.json'
source_name = 'ShapeNetV2'
data_dir = "/mnt/g/Shared drives/Deep Elastics/Shen Projects/Github/DeepSDF/data"
source_dir = '/mnt/g/Shared drives/Dataset_ShapeNetCore/data/ShapeNetCore.v2/'
subdir = 'SdfSamples'
dest_dir = os.path.join(data_dir, subdir, source_name)

surface_sampling = False
n_points = 500000
sigma = 0.0025

if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

if not os.path.isdir(os.path.join(data_dir, subdir)):
    os.mkdir(os.path.join(data_dir, subdir))

if not os.path.isdir(dest_dir):
    os.mkdir(dest_dir)

with open(split_filename) as f:
    split = json.load(f)

class_directories = split[source_name]
for class_dir in class_directories:
  class_path = os.path.join(source_dir, class_dir)
  instance_dirs = class_directories[class_dir]
  target_dir = os.path.join(dest_dir, class_dir)
  if not os.path.isdir(target_dir):
    os.mkdir(target_dir)
  for instance_dir in instance_dirs:
    shape_dir = os.path.join(class_path, instance_dir)
    if os.path.isdir(shape_dir):
      processed_filepath = os.path.join(target_dir, instance_dir)
      if os.path.isdir(processed_filepath):
        logging.debug("skipping " + processed_filepath)
        print("skipping " + processed_filepath)
        continue
      else:
        os.mkdir(processed_filepath)

      try:
        mesh_filename = deep_sdf.data.find_mesh_in_directory(shape_dir)
        mesh = trimesh.load(mesh_filename)
        # points, fid = trimesh.sample.sample_surface(mesh, count=n_points)
        # noise = np.random.normal(0, sigma, points.shape)
        # noisy_points = points + noise
        points, sdf = sample_sdf_near_surface(mesh, 500000)
        np.save(os.path.join(processed_filepath, 'points.npy'), points)
        np.save(os.path.join(processed_filepath, 'sdf.npy'), sdf)

      except deep_sdf.data.NoMeshFileError:
        logging.warning("No mesh found for instance " + instance_dir)
      except deep_sdf.data.MultipleMeshFileError:
        logging.warning("Multiple meshes found for instance " + instance_dir)
    

# for file_name in data:
#     # full_file_name = os.path.join('/content/drive/Shareddrives/Dataset_ShapeNetCore/data/ShapeNetCore.v2/03001627/', file_name + '/models/model_normalized.obj')
#     if os.path.isdir(file_name):
#       mesh = trimesh.load('model_normalized.obj')
#       points, sdf = sample_sdf_near_surface(mesh, number_of_points=500000)





libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)



GLError: GLError(
	err = 12289,
	baseOperation = eglInitialize,
	cArguments = (
		<OpenGL._opaque.EGLDisplay_pointer object at 0x7f35370a0d40>,
		c_long(0),
		c_long(0),
	),
	result = 0
)

Categories
```
04379243 table 
03710193 mailboxv
04530566 watercraft
03790512 motorbike
03207941 dishwasher
02828884 bench
03948459 pistol
04099429 rocket
03691459 loudspeaker
03337140 file cabinet
02773838 bag 
02933112 cabinet
02818832 bed
02843684 birdhouse
03593526 jar 
03211117 display
03928116 piano
03261776 earphone
04401088 telephone
04225987 skateboard 
04330267 stove
03759954 microphone
02924116 bus
03797390 mug
02958343 car
04074963 remote
02808440 bathtub
02880940 bowl
03085013 keyboard
02876657 bottle
03467517 guitar
04554684 washer
02834778 bicycle
03325088 faucet
04460130 tower
04004475 printer
02954340 cap
03001627 chair
02871439 bookshelf
02942699 camera
02691156 airplane
03642806 laptop
02801938 basket
04256520 sofa
03624134 knife
02946921 can
04090263 rifle
04468005 train
03938244 pillow
03636649 lamp
02747177 trash bin
```

